In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 33.7 MB/s eta 0:00:00


In [ ]:
import requests
from transformers import FuyuForCausalLM, AutoProcessor, BitsAndBytesConfig
from PIL import Image
import torch

model_id = 'ybelkada/fuyu-8b-sharded'
processor = AutoProcessor.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/192 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/848 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/6.13M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

In [ ]:
quantization_config = BitsAndBytesConfig(
	load_in_4bit=True,
	bnb_4bit_compute_dtype=torch.float16
)
model = FuyuForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/60.5k [00:00<?, ?B/s]

model-00001-of-00011.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

model-00002-of-00011.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

model-00003-of-00011.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

...

model-00009-of-00011.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

model-00010-of-00011.safetensors:   0%|          | 0.00/2.15G [00:00<?, ?B/s]

model-00011-of-00011.safetensors:   0%|          | 0.00/22.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

In [ ]:
import requests
img_url = 'https://ugc-media.4gamers.com.tw/puku-prod-zh/member/Rf1kF4Ov88yuaM7c4hYnn8B86zE/mondou/08c272cd-d0cf-491b-b0fb-6fa0519535b7.jpg'
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')

text_prompt = 'Generate a coco-style caption.\n'

model_inputs = processor(text=text_prompt, images=raw_image)
for k, v in model_inputs.items():
    if isinstance(v, torch.Tensor):
        model_inputs[k] = v.to('cuda')

In [ ]:
import requests

def get_text(img_url, text_prompt):
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
    model_inputs = processor(text=text_prompt, images=raw_image)
    for k, v in model_inputs.items():
        if isinstance(v, torch.Tensor):
            model_inputs[k] = v.to('cuda')
    generation_output = model.generate(**model_inputs, max_new_tokens=7)
    generation_text = processor.batch_decode(generation_output[:, -7:], skip_special_tokens=True)
    return generation_text

In [ ]:
img_url1 = 'https://ugc-media.4gamers.com.tw/puku-prod-zh/member/Rf1kF4Ov88yuaM7c4hYnn8B86zE/mondou/08c272cd-d0cf-491b-b0fb-6fa0519535b7.jpg'
text_prompt1 = 'Generate a coco-style caption.\n'
generation_text = get_text(img_url1, text_prompt1)
generation_text

Setting `pad_token_id` to `eos_token_id`:71013 for open-end generation.


['An aerial view of the pyramid and the beach.']